In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np


### Encode for RNN, RNN return encoded prediction then we decode it.

In [2]:
alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
# char_to_int = list(enumerate(alphabets))


In [3]:
# new = {}
# for i,v in int_to_char.items():
#     new[v] = i
# new   
int_to_char = dict(list(enumerate(alphabets)))
char_to_int = dict((v,i) for i,v in enumerate(alphabets))
char_to_int['X']


23

#### Prepare the dataset

In [4]:
x = []
y = []
seq_length = 1

for i in range(len(alphabets)-seq_length):
    seq_in = [alphabets[i]]
    seq_out = alphabets[i+1]
    print(seq_in , '-->', seq_out)
    x += [[char_to_int[seq_in[0]]]]
    y += [char_to_int[seq_out]]
y

['A'] --> B
['B'] --> C
['C'] --> D
['D'] --> E
['E'] --> F
['F'] --> G
['G'] --> H
['H'] --> I
['I'] --> J
['J'] --> K
['K'] --> L
['L'] --> M
['M'] --> N
['N'] --> O
['O'] --> P
['P'] --> Q
['Q'] --> R
['R'] --> S
['S'] --> T
['T'] --> U
['U'] --> V
['V'] --> W
['W'] --> X
['X'] --> Y
['Y'] --> Z


[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25]

In [5]:
x

[[0],
 [1],
 [2],
 [3],
 [4],
 [5],
 [6],
 [7],
 [8],
 [9],
 [10],
 [11],
 [12],
 [13],
 [14],
 [15],
 [16],
 [17],
 [18],
 [19],
 [20],
 [21],
 [22],
 [23],
 [24]]

In [6]:
x = np.reshape(x, (25,1,1))
x

array([[[ 0]],

       [[ 1]],

       [[ 2]],

       [[ 3]],

       [[ 4]],

       [[ 5]],

       [[ 6]],

       [[ 7]],

       [[ 8]],

       [[ 9]],

       [[10]],

       [[11]],

       [[12]],

       [[13]],

       [[14]],

       [[15]],

       [[16]],

       [[17]],

       [[18]],

       [[19]],

       [[20]],

       [[21]],

       [[22]],

       [[23]],

       [[24]]])

#### we divded by 25 becasue in minmax scaling min is 0 hence we can directly divide by 25 and get a normalized data

#### why 25 is used instead of 24?
<p style='color:green'> because y has 25 and not x

In [8]:
x_norm = x / 25
x_norm

array([[[0.  ]],

       [[0.04]],

       [[0.08]],

       [[0.12]],

       [[0.16]],

       [[0.2 ]],

       [[0.24]],

       [[0.28]],

       [[0.32]],

       [[0.36]],

       [[0.4 ]],

       [[0.44]],

       [[0.48]],

       [[0.52]],

       [[0.56]],

       [[0.6 ]],

       [[0.64]],

       [[0.68]],

       [[0.72]],

       [[0.76]],

       [[0.8 ]],

       [[0.84]],

       [[0.88]],

       [[0.92]],

       [[0.96]]])

In [14]:
from keras.utils import to_categorical
a = [1,2,3,1,2,3]
to_categorical(a)

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [15]:
y_new = to_categorical(y)
y_new

array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 

In [17]:
# output variable is y_new
y_new.shape, x_norm.shape

((25, 26), (25, 1, 1))

# Build the model

In [21]:
from keras.layers import SimpleRNN, Dense, Input
from keras.models import Sequential

model = Sequential()
model.add(Input(shape=(1,1)))
model.add(Dense(25,activation='relu'))
model.add(SimpleRNN(32)) #node size 32
model.add(Dense(26,activation='softmax'))

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 1, 25)          │            50 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_2 (SimpleRNN)        │ (None, 32)             │         1,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 26)             │           858 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,764 (10.80 KB)

 Trainable params: 2,764 (10.80 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
#### complie
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

In [24]:
model.fit(x_norm, y_new, epochs=500, batch_size=1)

Epoch 1/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9600 - loss: 1.0368   
Epoch 2/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9600 - loss: 1.0359
Epoch 3/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9600 - loss: 1.0288 
Epoch 4/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9600 - loss: 1.0303 
Epoch 5/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9600 - loss: 1.0273 
Epoch 6/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9600 - loss: 1.0265
Epoch 7/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9600 - loss: 1.0255 
Epoch 8/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9600 - loss: 1.0227 
Epoch 9/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9600 - loss: 1.0220
Epoch 10/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9200 - loss: 1.0238
Epoch 11/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9600 - loss: 1.0197 
Epoch 12/500
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - 

In [27]:
model.evaluate(x_norm, y_new, batch_size=1)

25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 1.0000 - loss: 0.3983  


[0.3982647657394409, 1.0]

In [50]:
new = 'a'
new = char_to_int[new.upper()] / 25
new = np.reshape(new, (1,1,1))
num = model.predict(new, verbose=False).argmax(axis=1)
int_to_char[num[0]]

'B'

## Length 3

In [78]:
x = []
y = []
seq_length = 3

for i in range(len(alphabets)-seq_length):
    seq_in = alphabets[i:i+seq_length]
    seq_out = alphabets[i+seq_length]
    x.append([char_to_int[char] for char in seq_in])
    y += [char_to_int[seq_out]]


x = np.reshape(x, (23,seq_length,1))
x_norm = x / 25
y_new = to_categorical(y)
y_new.shape


(23, 26)

In [80]:
from keras.layers import SimpleRNN, Dense, Input
from keras.models import Sequential

model = Sequential()
model.add(Input(shape=(3,1)))
model.add(Dense(23,activation='relu'))
model.add(SimpleRNN(32)) #node size 32
model.add(Dense(26,activation='softmax'))

model.summary()
#### complie
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 3, 23)          │            46 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_5 (SimpleRNN)        │ (None, 32)             │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 26)             │           858 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,696 (10.53 KB)

 Trainable params: 2,696 (10.53 KB)

 Non-trainable params: 0 (0.00 B)

In [81]:

model.fit(x_norm, y_new, epochs=500, batch_size=1)

Epoch 1/500


23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.0000e+00 - loss: 3.3394
Epoch 2/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 3.2752
Epoch 3/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0000e+00 - loss: 3.2474
Epoch 4/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0435 - loss: 3.2206   
Epoch 5/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0435 - loss: 3.1944   
Epoch 6/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0435 - loss: 3.1662     
Epoch 7/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0435 - loss: 3.1456    
Epoch 8/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.0435 - loss: 3.1205     
Epoch 9/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0435 - loss: 3.0936     
Epoch 10/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0435 - loss: 3.0687   
Epoch 11/500
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 3.0407 
Epoch 12/500
23/23 ━━━━━━━━━━━

In [82]:
new = 'JKL'
new = [char_to_int[i.upper()]/25 for i in new]
new = np.reshape(new, (1,3,1))
num = model.predict(new, verbose=False).argmax(axis=1)
int_to_char[num[0]]

'M'

In [ ]:
lstm airplane/bus
gru sonu sood